Importing required libraries

In [1]:
import os
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np
from tqdm import tqdm

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torchvision.transforms as transforms

Creating dataset

In [2]:
class ImageDataset(Dataset) :

    def __init__(self, transform) :
        self.root_path = 'PACS/kfold/'

        # Listing the domains
        self.domains = os.listdir(self.root_path)

        # Listing the classes 
        self.classes = os.listdir(self.root_path+'cartoon')

        # Transformations
        self.transforms = transform

        self.images = []
        self.domains_y = []
        self.classes_y = []

        for i_dom, domain in enumerate(self.domains) :
            for i_cla, cla in enumerate(self.classes) :
                for image in os.listdir(self.root_path+domain+'/'+cla) :
                    # Finding image path
                    image_path = self.root_path+domain+'/'+cla+'/'+image
                    img = cv2.imread(image_path)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    self.images.append(img)
                    self.domains_y.append(i_dom)
                    self.classes_y.append(i_cla)

        self.images = np.array(self.images)
        self.domains_y = np.array(self.domains_y)
        self.classes_y = np.array(self.classes_y)

    def __getitem__(self, index) :
        return self.transforms(self.images[index]), self.domains_y[index], self.classes_y[index]

    def __len__(self) :
        return len(self.images)

Defining transforms

In [3]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

In [6]:
dataset = ImageDataset(transform=transform)

In [7]:
# Train and test split
train_dataset, test_dataset = random_split(dataset, [7000, 2991])

In [8]:
train_dataloader = DataLoader(dataset=dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(dataset=dataset, batch_size=4, shuffle=True)

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim